In [1]:
import pandas as pd
import gensim
from sklearn.linear_model import LinearRegression
import numpy as np

In [2]:
df = pd.read_csv('anew2017_data.csv', usecols = [0,2,3,4,5,6,7])
rating_matrix = df.set_index('word').T.to_dict(orient='list')
df.head()

,word,valence,valence_sd,arousal,arousal_sd,dominance,dominance_sd
0,abandon,2.36,1.65,4.66,2.61,3.45,2.44
1,abduction,2.76,2.06,5.53,2.43,3.49,2.38
2,able,6.74,2.00,4.30,2.17,6.83,2.04
3,abortion,3.50,2.30,5.39,2.80,4.59,2.54
4,abrupt,4.22,1.50,4.95,2.16,4.78,1.88


In [3]:
# http://mccormickml.com/2016/04/12/googles-pretrained-word2vec-model-in-python/
# https://stackoverflow.com/questions/50478046/memory-error-when-using-gensim-for-loading-word2vec

In [4]:
wv_model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True, limit=999900)

In [5]:
wvec_words = set(wv_model.vocab.keys())
senti_words = set(rating_matrix.keys())
common_words = wvec_words.intersection(senti_words)
len(common_words), len(wvec_words), len(senti_words)

(3181, 999900, 3188)

In [7]:
dataX = [wv_model[word] for word in common_words]
dataY = [rating_matrix[word] for word in common_words]
dataX = np.array(dataX)
dataY = np.array(dataY)
dataY.shape, dataX.shape

((3181, 6), (3181, 300))

In [8]:
regrModel= LinearRegression()
regrModel.fit(dataX,dataY)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [12]:
twords = ['abrupt', 'absolute', 'bargain', 'clobber', 'home', 'premium', 'baseball', 'stink', 'home']
for tword in twords:
    if tword in rating_matrix:
        test_x = wv_model[tword]
        test_y = rating_matrix[tword]
        pred_y = regrModel.predict(test_x.reshape(1,-1))
        rmse = np.sqrt(((pred_y - test_y)**2).mean())
        print ("{0:.4f} is the RMS error for {1}".format(rmse, tword))
    else:
        pass
        #print ("\"{}\" not found in ANEW dataset".format(tword))

0.2860 is the RMS error for abrupt
0.3683 is the RMS error for bargain
0.7115 is the RMS error for home
0.3406 is the RMS error for baseball
0.4422 is the RMS error for stink
0.7115 is the RMS error for home
